# BERT Model:
https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 
     |████████████████████████████████| 6.5 MB 38.7 MB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


import pandas as pd 
import numpy as np


df = pd.read_csv('labeled_threehundredtweets.csv')
df.head()

,label,Datetime,Text,Username,UserLocation,ReplyCount,LikeCount,RetweetCount
0,1,2022-02-24 18:20:02+00:00,Construction on #AllTrains at New York Penn St...,511NY,New York State,0,0,0
1,1,2022-02-24 10:04:16+00:00,Cleared: Construction on #ELine SB from 23 St ...,511nyACE,New York City,0,0,0
2,1,2022-02-24 10:04:16+00:00,Cleared: Construction on #ELine SB from 23 St ...,511NY,New York State,0,0,0
3,1,2022-02-24 02:43:12+00:00,Construction on #ELine SB from 23 St to Spring...,511nyACE,New York City,0,0,0
4,1,2022-02-24 02:43:11+00:00,Construction on #ELine SB from 23 St to Spring...,511NY,New York State,0,0,0


In [ ]:
df = df.drop(['UserLocation', 'Datetime', 'Username', 'ReplyCount', 'LikeCount', 'RetweetCount'], axis=1)
df.head()

,label,Text
0,1,Construction on #AllTrains at New York Penn St...
1,1,Cleared: Construction on #ELine SB from 23 St ...
2,1,Cleared: Construction on #ELine SB from 23 St ...
3,1,Construction on #ELine SB from 23 St to Spring...
4,1,Construction on #ELine SB from 23 St to Spring...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   301 non-null    int64 
 1   Text    301 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.8+ KB


In [ ]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{0: 1, 1: 0}

In [ ]:
df['Text'].value_counts()


Updated: Construction on #M5Bus NB at Av Of The Americas/W 54 St https://t.co/0WhyqmTe2n                                                                                                                                                                                                                             2
Cleared: Construction on #ELine SB from 23 St to Spring St                                                                                                                                                                                                                                                           2
Cleared: Construction on #CLine Both directions from 59 St - Columbus Circle to W 4 St - Wash Sq                                                                                                                                                                                                                     2
Updated: Construction on #B70Bus SB at 37 St/4 Av https://t.co/n58D

In [ ]:
#df['Text'] = df.Text.replace(label_dict)
df

,label,Text
0,1,Construction on #AllTrains at New York Penn St...
1,1,Cleared: Construction on #ELine SB from 23 St ...
2,1,Cleared: Construction on #ELine SB from 23 St ...
3,1,Construction on #ELine SB from 23 St to Spring...
4,1,Construction on #ELine SB from 23 St to Spring...
...,...,...
296,1,"In the #Bronx, southbound #BronxRiverParkway h..."
297,1,"Accident, center lane blocked in #BrooklynOnTh..."
298,0,"We are on the Expressway to #WrestleMania, and..."
299,0,We congratulate Gavaskar Reid on the 6th year ...


# **Train and Validation Split**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Text', 'label', 'data_type']).count()

,,
Text,label,data_type
"""The limitless imagination of these genres hit a roadblock when it comes to envisioning other political systems."" #SciFi #Fantasy\nhttps://t.co/JOFz666qGS",0,train
#Advertising #OnlineSales Imitate these prestigious traffic building solutions to gain unrestricted visitors to impatiently contact concerning any business-at-home https://t.co/o3O6FdUvDW https://t.co/rDvf34N2k1,0,train
#Entrepreneurs #HomeBusiness This important terrific website traffic generation tactic will disclose ways to create a prospecting mania by using social websites https://t.co/lZP7yEPtyi https://t.co/aOOLDm0iPO,0,train
#HomeBusiness #Success Should you go here I am going to demonstrate a teaching video clip I've built to forever resolve your own difficulties regarding targeted traffic https://t.co/lZP7yEPtyi https://t.co/RyWDyzCBqO,0,train
#LeadGen #BlogPost New method of getting traffic https://t.co/XPWnHBFgtD https://t.co/jpYa9OuAfp,0,train
...,...,...
"“MBK helps me to help someone if they are feeling down. I would like to work in the construction field in the future.” -Isiah Grade 7, Edenwald School, Mount Pleasant Cottage School UFSD #MyHistoryMyFuture #NYSMBK #BlackHistoryMonth @NYSEDNews @NYSMBK https://t.co/U3zZdeBw8S",0,val
"“MBK helps me to help someone if they are feeling down. I would like to work in the construction field in the future.” -Isiah Grade 7, Edenwald School, Mount Pleasant Cottage School UFSD #MyHistoryMyFuture #NYSMBK #BlackHistoryMonth @NYSEDNews @EdTrustNY https://t.co/Hoi7cwKmg0",0,train
"“We’re going to have a shutdown,” Bob Bolus, the organizer, told The Epoch Times. “We’ll shut it down.”\n\nA US #TruckerConvoy planning to leave soon plans on shutting down traffic on the Capital Beltway outside #WashingtonDC. https://t.co/qvXAUbkNhc",1,val


# **Bert Tokenizer and Encoding**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(dataset_train)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')




In [ ]:
import random
import torch 

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERTnew_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/85 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.4403877028647591
Validation loss: 0.34718540380708873
F1 Score (Weighted): 0.843115051810704


Epoch 2:   0%|          | 0/85 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.1650680213344886
Validation loss: 0.4707365677750204
F1 Score (Weighted): 0.86651411136537


Epoch 3:   0%|          | 0/85 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.11443835043293589
Validation loss: 0.32337562179600354
F1 Score (Weighted): 0.9343147651425363


Epoch 4:   0%|          | 0/85 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.05283694501199267
Validation loss: 0.41513531797681935
F1 Score (Weighted): 0.9120401337792643


Epoch 5:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

model.load_state_dict(torch.load('finetuned_BERTnew_epoch_5.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_vals, predictions)

In [ ]:
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score
loo = LeaveOneOut()

scores = cross_val_score(model, dataset_train.attention_masks_train, dataset_train.labels_train, scoring = "f1_weighted",
                         cv=loo, n_jobs=-1)
mean(scores)